In [66]:
# Base de Conocimiento
KB = {
    "Hechos": [
        
    ],
    "Reglas": [
        {
            "condicion": "riesgo=bajo && eficiencia_energetica=True", 
            "decision": "Aprobado"
        },
        {
            "condicion": "riesgo=alto && impacto_ambiental=negativo",
            "decision": "Rechazado"
        },
        {
            "condicion": "endeudamiento=medio && impacto_ambiental=moderado",
            "decision": "AprobarCondicionadoGarantia"
        }
    ],
    "ReglasCombinadas": [
        {
            # ESCOGER SOLAMENTE DOS HECHOS?
            "condicion": "riesgo=medio && impacto_ambiental=positivo && cofinanciamiento=True", 
            "decision": "AprobadoCondicionadoMitigaciónSeguimiento"
        },
        {
            "condicion": "riesgo=bajo && impacto_ambiental=moderado && endeudamiento=medio",
            "decision": "AprobadoCondicionadoSeguimientoAmbiental"
        },
        {
            "condicion": "riesgo=medio && eficiencia_energetica=True && endeudamiento=bajo",
            "decision": "AprobadoConDescuentoTasa"
        },
        {
            "condicion": "riesgo=alto && eficiencia_energetica=False && impacto_ambiental=negativo",
            "decision": "RechazadoDefinitivo"
        },
        {
            "condicion": "riesgo=bajo && endeudamiento=alto && impacto_ambiental=positivo",
            "decision": "AprobadoCondicionadoReestructuracionDeuda"
        },
        {
            "condicion": "riesgo=alto && sector_prioritario=True && impacto_ambiental=positivo",
            "decision": "RevisionEspecializadaSectorPrioritario"
        },
    
    ]
}

In [ ]:
def aplicar_reglas(solicitante, proyecto):
    hechos = hechos_de(solicitante, proyecto)
    hechos_por_condicion = {h["condicion"]: h["valor"] for h in hechos}

    decisiones = []
    justificaciones = []

    for regla in KB["Reglas"]:
        cond_str = regla.get("condicion", "")
        if cond_str and coincide_condicion(cond_str, hechos_por_condicion):
            decisiones.append(regla["decision"])
            justificaciones.append(cond_str)

    return decisiones, justificaciones


def coincide_condicion(condicion_str, hechos_por_condicion):
    for expresion in condicion_str.split("&&"):
        clave, valor_esperado = parsear_expresion(expresion)
        valor_real = hechos_por_condicion.get(clave)
        if valor_real is None or not comparar_valores(valor_real, valor_esperado):
            return False
    return True


def parsear_expresion(expresion):
    if "=" not in expresion:
        raise ValueError(f"Condición mal formada: {expresion}")
    clave, valor = expresion.split("=", 1)
    return clave.strip(), normalizar_valor(valor.strip())


def normalizar_valor(valor):
    texto = valor.lower()
    if texto == "true":
        return True
    if texto == "false":
        return False
    try:
        return int(valor)
    except ValueError:
        try:
            return float(valor)
        except ValueError:
            return texto  # compara strings en minúsculas


def comparar_valores(valor_real, valor_esperado):
    if isinstance(valor_real, str) and isinstance(valor_esperado, str):
        return valor_real.strip().lower() == valor_esperado.strip().lower()
    return valor_real == valor_esperado


In [68]:
# Aplicación de Reglas Combinadas
def aplicar_reglas_combinadas(solicitante, proyecto):
    hechos_por_condicion = {
        hecho["condicion"]: hecho["valor"]
        for hecho in hechos_de(solicitante, proyecto)
    }

    decisiones = []
    for regla in KB.get("ReglasCombinadas", []):
        condicion = regla.get("condicion")
        if condicion and coincide_condicion(condicion, hechos_por_condicion):
            decisiones.append(regla["decision"])
    return decisiones


In [69]:
# Funciones Auxiliares
def verificar_condicion(hechos, condicion):
    for condic in condicion:
        if condic not in hechos or not hechos[condic]:
            return False
    return True

def verificar_factores(factores_adicionales, factores_requeridos):
    for factor in factores_requeridos:
        if factor not in factores_adicionales:
            return False
    return True

In [70]:

def agregar_hecho(solicitante, proyecto, condicion, valor):
    KB["Hechos"].append({
        "solicitante": solicitante,
        "proyecto": proyecto,
        "condicion": condicion,
        "valor": valor
    })

In [71]:
def hechos_de(solicitante, proyecto):
    return [h for h in KB["Hechos"] if h["solicitante"] == solicitante and h["proyecto"] == proyecto]

In [ ]:
'''
def aplicar_reglas(solicitante, proyecto):
    hechos = hechos_de(solicitante, proyecto)
    decisiones = []
    justificaciones = []
    for regla in KB["Reglas"]:
        condiciones = regla["condicion"].split("&&")
        cumple = True
        for cond in condiciones:
            cond = cond.strip()
            if "=" in cond:
                clave, valor = cond.split("=")
                clave = clave.strip()
                valor = valor.strip()
                if valor == "True":
                    valor = True
                elif valor == "False":
                    valor = False
                encontrado = any(
                    h["condición"] == clave and h["valor"] == valor
                    for h in hechos
                )
                if not encontrado:
                    cumple = False
                    break
        if cumple:
            decisiones.append(regla["decision"])
            justificaciones.append(regla["condición"])
    return decisiones, justificaciones
'''

In [73]:
def priorizar_recomendaciones(decisiones):
    prioridad = {
        "Aprobado": 3,
        "AprobadoCondicionadoMitigaciónSeguimiento": 2,
        "Rechazado": 0
    }
    if not decisiones:
        return "Sin decision"
    return sorted(decisiones, key=lambda d: prioridad.get(d, -1), reverse=True)[0]

In [74]:
def evaluar_credito(solicitante, proyecto):
    print(f"Evaluando crédito para {solicitante} y proyecto {proyecto}")
    decisiones, justificaciones = aplicar_reglas(solicitante, proyecto)
    decision_final = priorizar_recomendaciones(decisiones)
    print(f"Decision priorizada: {decision_final}")
    print(f"Justificación: {justificaciones}")
    print("---")

In [75]:
agregar_hecho("Juan", "Edificio Verde", "riesgo", "bajo")
agregar_hecho("Juan", "Edificio Verde", "eficiencia_energetica", True)

agregar_hecho("Ana", "Fabrica", "riesgo", "alto")
agregar_hecho("Ana", "Fabrica", "impacto_ambiental", "negativo")

agregar_hecho("Luis", "Planta Solar", "riesgo", "medio")
agregar_hecho("Luis", "Planta Solar", "impacto_ambiental", "positivo")
agregar_hecho("Luis", "Planta Solar", "cofinanciamiento", True)

In [76]:
evaluar_credito("Juan", "Edificio Verde")
evaluar_credito("Ana", "Fabrica")
evaluar_credito("Luis", "Planta Solar")

Evaluando crédito para Juan y proyecto Edificio Verde


KeyError: 'condición'